原文地址：[itchat+pillow实现微信好友头像爬取和拼接](https://zhuanlan.zhihu.com/p/25782937?group_id=826931026017292288)

In [1]:
#我的Python版本是：
import sys  
print (sys.version)   
print (sys.version_info) 

3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)


这里我用Python2.7，实际上代码在Python3.6,3.5运行一切正常
# 核心
- itchat读取微信好友列表和头像
- 用pillow拼接头像画图

登录微信网页版，`itchat.auto_login()`会生成QR码，使用手机扫一扫即可登录。

提示`Please scan the QR code to log in.`时，用手机扫描弹出的二维码

[命令行二维码技巧：](https://itchat.readthedocs.io/zh/latest/#_4)
1. 通过以下命令可以在登陆的时候使用命令行显示二维码：`itchat.auto_login(enableCmdQR=True)`
2. 若背景色为浅色（白色），可以将enableCmdQR赋值为负值：`itchat.auto_login(enableCmdQR=-1)`


In [2]:
import itchat
itchat.auto_login()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as greedisgood


使用`friends`储存好友列表，`update=True`可以确保好友列表是最新的。注意好友列表第0个是自己

In [3]:
friends = itchat.get_friends(update=True)[0:]

`friends`好友列表第0个是自己，我们可以看一下。顺带说一下，好友列表的顺序 **（貌似）** 是按照好友添加顺序

In [5]:
friends[0]

{'UserName': '@1f44203b640865173ce49026d9dfbd3109eedd4a0dd753863eb5bd6ed16f56b1',
 'City': '',
 'DisplayName': '',
 'PYQuanPin': '',
 'RemarkPYInitial': '',
 'Province': '',
 'KeyWord': '',
 'RemarkName': '',
 'PYInitial': '',
 'EncryChatRoomId': '',
 'Alias': '',
 'Signature': '我买几个橘子去。你就在此地，不要走动。',
 'NickName': 'greedisgood',
 'RemarkPYQuanPin': '',
 'HeadImgUrl': '/cgi-bin/mmwebwx-bin/webwxgeticon?seq=18888205&username=@1f44203b640865173ce49026d9dfbd3109eedd4a0dd753863eb5bd6ed16f56b1&skey=@crypt_1e355562_cbb32ae58ba3109f695ccf972f500980',
 'UniFriend': 0,
 'Sex': 1,
 'AppAccountFlag': 0,
 'VerifyFlag': 0,
 'ChatRoomId': 0,
 'HideInputBarFlag': 0,
 'AttrStatus': 0,
 'SnsFlag': 1,
 'MemberCount': 0,
 'OwnerUin': 0,
 'ContactFlag': 0,
 'Uin': 1601516715,
 'StarFriend': 0,
 'Statues': 0,
 'MemberList': [],
 'WebWxPluginSwitch': 0,
 'HeadImgFlag': 1}

创建一个目录，用来保存所有好友头像。注意使用`os.chdir(user)`切换个到工作目录，方便后续保存图片。



In [5]:
import os
user = friends[0]["PYQuanPin"][0:]
print(user)
os.mkdir(user)
os.chdir(user)
os.getcwd()

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: ''

批量下载好友头像，储存到`friends[i]['img']`中。然后我们`print(friends[0]`看看有没有变化（正常情况下应该可以看到增加了img，以二进制方式储存头像）。因为我家网络经常链接失败，所以用`try...except...`来写这一段。

`"UserName"`这个字段开头总有个@符号，直接暴力去除。如果不喜欢的话，可以把`"UserName"`换成`"PYQuanPin"`，不过不保证重名。

同时把头像保存在`user`目录地下，方便下一步使用。

### python使用open经常报错：TypeError: an integer is required的解决方案
错误是由于从os模块引入了所有的函数导致的，os模块下有一个open函数，接受整型的文件描述符和打开模式，`from os import *`引入os模块的open函数，覆盖了python内建的open函数，导致错误。删除`from os import *`这行，然后再根据需要，指定引入os模块下的函数

In [7]:
for i in friends:
    try:
        i['img'] = itchat.get_head_img(userName=i["UserName"])
        i['ImgName']=i["UserName"][1:] + ".jpg"
    except ConnectionError:
        print('get '+i["UserName"][1:]+' fail')
    fileImage=open(i['ImgName'],'wb')
    fileImage.write(i['img'])
    fileImage.close()
#这里不建议看friends[0]，太长了

看看我有多少个好友（friends里面有多少条记录），看看下载了多少头像（`os.listdir(os.getcwd())`看目录底下有多少文件）

In [8]:
friendsSum=len(friends)
imgList=os.listdir(os.getcwd())
numImages=len(imgList)
print('I have ',friendsSum,'friend(s), and I got ',numImages,'image(s)')

I have  341 friend(s), and I got  347 image(s)


单个图像边长`eachsize=64`像素，一行`eachline=int(sqrt(numImages))+1`个头像，最终图像边长`eachSize*eachline`

In [9]:
import math
eachSize=64
eachLine=int(math.sqrt(numImages))+1
print("单个图像边长",eachSize,"像素，一行",eachLine,"个头像，最终图像边长",eachSize*eachLine)

单个图像边长 64 像素，一行 19 个头像，最终图像边长 1216


import图像处理Python图像处理库：PIL中Image
1. 新建一块画布
2. 在坐标（0,0）位置放上第一个人头像
3. 向右平移坐标

In [10]:
import PIL.Image as Image
toImage = Image.new('RGBA', (eachSize*eachLine,eachSize*eachLine))#新建一块画布
x = 0
y = 0
for i in imgList:
    try:
        img = Image.open(i)#打开图片
    except IOError:
        print("Error: 没有找到文件或读取文件失败",i)
    else:
        img = img.resize((eachSize, eachSize), Image.ANTIALIAS)#缩小图片
        toImage.paste(img, (x * eachSize, y * eachSize))#拼接图片
        x += 1
    if x == eachLine:
        x = 0
        y += 1
print("图像拼接完成")

Error: 没有找到文件或读取文件失败 .ipynb_checkpoints
Error: 没有找到文件或读取文件失败 2c5a6e6f32b1d30220f522cd56c7a2cd4f89955378284f1967a48c071b17fc31.jpg
Error: 没有找到文件或读取文件失败 bd3dc193f560b73c8d4a0c9bdb54520faf23e76d196a75f24a60c5fd90aebc29.jpg
Error: 没有找到文件或读取文件失败 README.md
Error: 没有找到文件或读取文件失败 requirements.txt
Error: 没有找到文件或读取文件失败 screenshots
Error: 没有找到文件或读取文件失败 wxImage.ipynb
Error: 没有找到文件或读取文件失败 wxImage.py
图像拼接完成


看一下拼接好的图像是什么样的（注意文件过大是常有的现象，请先去掉注释）
回到上一级目录（没人想在一堆文件里面找拼图吧？）
然后保存文件，顺带发送给文件传输助手

In [11]:
#toImage.show()

os.chdir(os.path.pardir)
os.getcwd()

toImage.save(friends[0]["PYQuanPin"][0:]+".jpg")
itchat.send_image(friends[0]["PYQuanPin"][0:]+".jpg", 'filehelper')

ValueError: unknown file extension: 

LOG OUT!


至此，大功告成，别忘记退出网页版微信

In [ ]:
itchat.logout()

In [8]:
import itchat
import os

import PIL.Image as Image
from os import listdir
import math

itchat.auto_login()

friends = itchat.get_friends(update=True)[0:]

user = friends[0]["NickName"]

print(user)

os.mkdir(user)

num = 0

for i in friends:
	img = itchat.get_head_img(userName=i["UserName"])
	fileImage = open(user + "/" + str(num) + ".jpg",'wb')
	fileImage.write(img)
	fileImage.close()
	num += 1

pics = listdir(user)

numPic = len(pics)

print(numPic)

eachsize = int(math.sqrt(float(640 * 640) / numPic))

print(eachsize)

numline = int(640 / eachsize)

toImage = Image.new('RGB', (640, 640))


print(numline)

x = 0
y = 0

for i in pics:
	try:
		#打开图片
		img = Image.open(user + "/" + i)
	except IOError:
		print("Error: 没有找到文件或读取文件失败")
	else:
		#缩小图片
		img = img.resize((eachsize, eachsize), Image.ANTIALIAS)
		#拼接图片
		toImage.paste(img, (x * eachsize, y * eachsize))
		x += 1
		if x == numline:
			x = 0
			y += 1


toImage.save(user + ".jpg")


itchat.send_image(user + ".jpg", 'filehelper')


itchat has already logged in.


greedisgood
341
34
18
Error: 没有找到文件或读取文件失败
Error: 没有找到文件或读取文件失败


<ItchatReturnValue: {'BaseResponse': {'Ret': 0, 'ErrMsg': '请求成功', 'RawMsg': '请求成功'}, 'MsgID': '2934680590321596128', 'LocalID': '15318978609091'}>